# Lab 4: Dropout, Gradient Clipping & Multitask w/ EarlyStopping

Simplified and cleaned code from your lab manual.

In [ ]:
# Lab 4a: Dropout & Gradient Clipping demo (synthetic data)
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Create synthetic dataset
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def create_dropout_model(dropout_rate=0.2):
    model = Sequential([
        Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
        Dropout(dropout_rate),
        Dense(32, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

dropout_model = create_dropout_model(0.2)
dropout_history = dropout_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=0)

def create_gradient_clip_model(clip_norm=1.0):
    model = Sequential([
        Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    optimizer = Adam(clipnorm=clip_norm)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

gradient_clip_model = create_gradient_clip_model(clip_norm=1.0)
gradient_clip_history = gradient_clip_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=0)

# Plot accuracies
plt.figure(figsize=(10,6))
plt.plot(dropout_history.history['accuracy'], label='Dropout Train Acc', linestyle='--')
plt.plot(dropout_history.history['val_accuracy'], label='Dropout Val Acc')
plt.plot(gradient_clip_history.history['accuracy'], label='GradClip Train Acc', linestyle='--')
plt.plot(gradient_clip_history.history['val_accuracy'], label='GradClip Val Acc')
plt.title('Training and Validation Accuracy'); plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend(); plt.grid(True); plt.show()

In [ ]:
# Lab 4b: Multitask learning with EarlyStopping on MNIST (parity task)
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load MNIST and create parity auxiliary task (even/odd)
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0; x_test = x_test / 255.0
y_train_parity = np.array([np.sum(list(map(int, str(y)))) % 2 for y in y_train])
y_test_parity = np.array([np.sum(list(map(int, str(y)))) % 2 for y in y_test])

# Display sample images
plt.figure(figsize=(6,6))
for i in range(9):
    plt.subplot(3,3,i+1); plt.xticks([]); plt.yticks([]); plt.grid(False)
    plt.imshow(x_train[i], cmap='gray'); plt.xlabel(y_train[i])
plt.show()

inputs = keras.Input(shape=(28,28))
x = layers.Flatten()(inputs)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)
outputs_parity = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=[outputs, outputs_parity])

model.compile(optimizer='adam',
              loss=['sparse_categorical_crossentropy', 'binary_crossentropy'],
              metrics=[['accuracy'], ['accuracy']])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(x_train, [y_train, y_train_parity], epochs=20, validation_split=0.2, callbacks=[early_stopping], verbose=0)

# Determine early stopping epoch (approx)
val_loss = history.history.get('val_loss', [])
if val_loss:
    early_epoch = np.argmin(val_loss) + 1
else:
    early_epoch = None

print('Early stopping occurred at epoch:', early_epoch)

# Plot training & validation loss with early stopping marker
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
if early_epoch:
    plt.axvline(x=early_epoch, color='r', linestyle='--', label='Early Stopping')
plt.xlabel('Epochs'); plt.ylabel('Loss'); plt.title('Training & Validation Loss'); plt.legend(); plt.show()

In [ ]:
# End of Lab 4